In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [12]:
%matplotlib inline

dataset = pd.read_csv('student-course1.csv', sep=';')

In [23]:
dataset.dtypes #check data types

studytime     int64
failures      int64
Dalc          int64
Walc          int64
health        int64
absences      int64
traveltime    int64
G1            int64
G2            int64
G3            int64
dtype: object

In [13]:
dataset = dataset[['studytime', 'failures', 'Dalc', 'Walc','health','absences','traveltime',
                   'G1', 'G2', 'G3']] # Chossing attributes that matter or affect the students results try different attributes
print(dataset)
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -3].values

     studytime  failures  Dalc  Walc  health  absences  traveltime  G1  G2  G3
0            2         0     1     1       3         6           2   5   6   6
1            2         0     1     1       3         4           1   5   5   6
2            2         3     2     3       3        10           1   7   8  10
3            3         0     1     1       5         2           1  15  14  15
4            2         0     1     2       5         4           1   6  10  10
..         ...       ...   ...   ...     ...       ...         ...  ..  ..  ..
390          2         2     4     5       4        11           1   9   9   9
391          1         0     3     4       2         3           2  14  16  16
392          1         3     3     3       3         3           1  10   8   7
393          1         0     3     4       5         0           3  11  12  10
394          1         0     3     3       5         5           1   8   9   9

[395 rows x 10 columns]


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=44)

In [39]:
class DeepNeuralNetwork():
    def __init__(self, sizes, epochs=15, l_rate=0.001):
        self.sizes = sizes
        self.epochs = epochs
        self.l_rate = l_rate

        # we save all parameters in the neural network in this dictionary
        self.params = self.initialization()

    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    def softmax(self, x, derivative=False):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)

    def initialization(self):
        # number of nodes in each layer
        input_layer=self.sizes[0]
        hidden_1=self.sizes[1]
        hidden_2=self.sizes[2]
        hidden_3=self.sizes[3]
        hidden_4=self.sizes[4]
        hidden_5=self.sizes[5]
        output_layer=self.sizes[6]

        params = {
            'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
            'b1':np.random.randn(hidden_1,),
            'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
            'b2':np.random.randn(hidden_2,),
            'W3':np.random.randn(hidden_3, hidden_2) * np.sqrt(1. / hidden_3),
            'b3':np.random.randn(hidden_3,),
            'W4':np.random.randn(hidden_4, hidden_3) * np.sqrt(1. / hidden_4),
            'b4':np.random.randn(hidden_4,),
            'W5':np.random.randn(hidden_5, hidden_4) * np.sqrt(1. / hidden_5),
            'b5':np.random.randn(hidden_5,),
            'W6':np.random.randn(output_layer, hidden_5) * np.sqrt(1. / output_layer),
            'b6':np.random.randn(output_layer,),
        }

        return params

    def forward_pass(self, x_train):
        params = self.params

        # input layer activations becomes sample
        params['A0'] = x_train

        # input layer to hidden layer 1
        params['Z1'] = np.dot(params["W1"], params['A0']) + params['b1']
        params['A1'] = self.sigmoid(params['Z1'])

        # hidden layer 1 to hidden layer 2
        params['Z2'] = np.dot(params["W2"], params['A1']) + params['b2']
        params['A2'] = self.sigmoid(params['Z2'])

         # hidden layer 2 to hidden layer 3
        params['Z3'] = np.dot(params["W3"], params['A2']) + params['b3']
        params['A3'] = self.sigmoid(params['Z3'])

         # hidden layer 3 to hidden layer 4
        params['Z4'] = np.dot(params["W4"], params['A3']) + params['b4']
        params['A4'] = self.sigmoid(params['Z4'])

        # hidden layer 4 to hidden layer 5
        params['Z5'] = np.dot(params["W5"], params['A4']) + params['b5']
        params['A5'] = self.sigmoid(params['Z5'])

        # hidden layer 5 to output_layer layer
        params['Z6'] = np.dot(params["W6"], params['A5'])  + params['b6']
        params['A6'] = self.softmax(params['Z6'])

        return params['A6']

    def backward_pass(self, y_train, output):


        params = self.params
        change_w = {}
        change_b = {}

       # Calculate W6 update
        error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z6'], derivative=True)
        change_w['W6'] = np.outer(error, params['A5'])
        change_b['B6'] = np.sum(error,axis=0,keepdims=True)
        # Calculate W5 update
        error = np.dot(params['W6'].T, error) * self.sigmoid(params['Z5'], derivative=True)
        change_w['W5'] = np.outer(error, params['A4'])
        change_b['B5'] = np.sum(error,axis=0,keepdims=True)
        # Calculate W4 update
        error = np.dot(params['W5'].T, error) * self.sigmoid(params['Z4'], derivative=True)
        change_w['W4'] = np.outer(error, params['A3'])
        change_b['B4'] = np.sum(error,axis=0,keepdims=True)
        # Calculate W3 update
        error = np.dot(params['W4'].T, error) * self.sigmoid(params['Z3'], derivative=True)
        change_w['W3'] = np.outer(error, params['A2'])
        change_b['B3'] = np.sum(error,axis=0,keepdims=True)
        # Calculate W2 update
        error = np.dot(params['W3'].T, error) * self.sigmoid(params['Z2'], derivative=True)
        change_w['W2'] = np.outer(error, params['A1'])
        change_b['B2'] = np.sum(error,axis=0,keepdims=True)
        # Calculate W1 update
        error = np.dot(params['W2'].T, error) * self.sigmoid(params['Z1'], derivative=True)
        change_w['W1'] = np.outer(error, params['A0'])
        change_b['B1'] = np.sum(error,axis=0,keepdims=True)
        return change_w
        return change_b

    def update_network_parameters(self, changes_to_w):

        for key, value in changes_to_w.items():
            self.params[key] -= self.l_rate * value

    
    
    
    
    
    
    
    def compute_accuracy(self, x_val, y_val):

        predictions = []

        for x, y in zip(x_val, y_val):
            output = self.forward_pass(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))

        return np.mean(predictions)

    def train(self, x_train, y_train, x_val, y_val):
        start_time = time.time()
        for iteration in range(self.epochs):
            for x,y in zip(x_train, y_train):
                output = self.forward_pass(x)
                changes_to_w = self.backward_pass(y, output)
                self.update_network_parameters(changes_to_w)

            accuracy = self.compute_accuracy(x_val, y_val)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                iteration+1, time.time() - start_time, accuracy * 100
            ))


In [40]:

dnn = DeepNeuralNetwork(sizes=[9,10,9,9,5,5,3])
dnn.train(x_train, y_train, x_test, y_test)  


Epoch: 1, Time Spent: 0.10s, Accuracy: 100.00%
Epoch: 2, Time Spent: 0.22s, Accuracy: 100.00%
Epoch: 3, Time Spent: 0.34s, Accuracy: 100.00%
Epoch: 4, Time Spent: 0.47s, Accuracy: 100.00%
Epoch: 5, Time Spent: 0.60s, Accuracy: 100.00%
Epoch: 6, Time Spent: 0.72s, Accuracy: 100.00%
Epoch: 7, Time Spent: 0.81s, Accuracy: 100.00%
Epoch: 8, Time Spent: 0.91s, Accuracy: 100.00%
Epoch: 9, Time Spent: 1.00s, Accuracy: 100.00%
Epoch: 10, Time Spent: 1.10s, Accuracy: 100.00%
Epoch: 11, Time Spent: 1.20s, Accuracy: 100.00%
Epoch: 12, Time Spent: 1.30s, Accuracy: 100.00%
Epoch: 13, Time Spent: 1.40s, Accuracy: 100.00%
Epoch: 14, Time Spent: 1.54s, Accuracy: 100.00%
Epoch: 15, Time Spent: 1.63s, Accuracy: 100.00%
